In [1]:
# Hello readers, I am Hangming Zhang, the author of "Joint Mitigation of Quantum Gate and Measurement Errors
# via the Z-mixed-state Expression of the Pauli Channel".
# I am delighted to share my code here.
# I conducted simulations using the Pennylane library, and the specific code is shown below.

In [2]:
import pennylane as qml
from pennylane import numpy as np

In [3]:
n_qubits = 4
dev = qml.device('default.mixed', wires=n_qubits)

In [4]:
# To obtain all Pauli basis matrices on N qubits,
# we first define a class in order to accomplish this task.
class NqubitsPauliMatrices:
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits
    
    def get_pauli_matrices_of_n_qubits(self):
        pauli_i_matrix_1_qubit = np.array([[1, 0], [0, 1]])
        pauli_x_matrix_1_qubit = np.array([[0, 1], [1, 0]])
        pauli_y_matrix_1_qubit = np.array([[0, -1j], [1j, 0]])
        pauli_z_matrix_1_qubit = np.array([[1, 0], [0, -1]])
        pauli_set_n_qubits = []
        pauli_set = []
        pauli_set_1_qubit = []
        pauli_set_1_qubit.append(pauli_i_matrix_1_qubit)
        pauli_set_1_qubit.append(pauli_x_matrix_1_qubit)
        pauli_set_1_qubit.append(pauli_y_matrix_1_qubit)
        pauli_set_1_qubit.append(pauli_z_matrix_1_qubit)
        for i in pauli_set_1_qubit:
            for j in pauli_set_1_qubit:
                temp = np.kron(i, j)
                pauli_set.append(temp)
        if self.n_qubits == 1:
            return pauli_set_1_qubit
        if self.n_qubits == 2:
            return pauli_set
        for _ in range(self.n_qubits - 2):
            for i in pauli_set_1_qubit:
                for j in pauli_set:
                    temp = np.kron(i, j)
                    pauli_set_n_qubits.append(temp)
            pauli_set = pauli_set_n_qubits
            pauli_set_n_qubits = []
        pauli_set_n_qubits = pauli_set.copy()
        return pauli_set_n_qubits
    

In [5]:
# After obtaining all Pauli bases on N qubits,
# we can define a class to obtain all Kraus matrices of a depolarizing channel on N qubits.
class NqubitsDepolarizingChannel:
    def __init__(self, n_qubits, pauli_set_n_qubits, p):
        self.n_qubits = n_qubits
        self.pauli_set_n_qubits = pauli_set_n_qubits
        self.p = p
    
    def get_kraus_matrices_of_a_depolarizing_channel(self):
        kraus_matrices = self.pauli_set_n_qubits.copy()
        for i in range(1, len(kraus_matrices)):
            kraus_matrices[i] = kraus_matrices[i] * np.sqrt((1 - self.p)/(4 ** self.n_qubits - 1))
        kraus_matrices[0] = np.sqrt(self.p) * kraus_matrices[0]
        return kraus_matrices

In [6]:
class QEMZMSEPC:
    def __init__(self, n_qubits,
                 nqubitspaulimatrices: NqubitsPauliMatrices,
                 nqubitsdepolarizingchannel: NqubitsDepolarizingChannel):
        self.n_qubits = n_qubits
        self.nqubitspaulimatrices = NqubitsPauliMatrices
        self.nqubitsdepolarizingchannel = NqubitsDepolarizingChannel
        
    
    def get_circuit(self, operations, paras, p, kraus_matrices_of_a_pauli_channel):
    # operations: 'RX' or 'RY' or 'RZ' or 'CNOT'
    # Example of operations: ['RX', 'CNOT', 'RY']
    # paras: Parameters associated with each operation in operations.
    # For rotation gates, these parameters include the wire and rotation angle,
    # while for CNOT gates, these parameters consist of the control qubit index and the target qubit index.
    # Example of paras: [[0, 0.5], [1, 0], [1, 1.6]]
    # kraus_matrices_of_a_pauli_channel: This is a custom Pauli channel on N qubits,
    # representing measurement noise.
    
        for i, operation in enumerate(operations):
            if operation == 'RX':
                qml.RX(paras[i][1], wires=paras[i][0])
            elif operation == 'RY':
                qml.RY(paras[i][1], wires=paras[i][0])
            elif operation == 'RZ':
                qml.RZ(paras[i][1], wires=paras[i][0])
            elif operation == 'CNOT':
                qml.CNOT(wires=[paras[i][0], paras[i][1]])
        
        nqubitspaulimatrices = self.nqubitspaulimatrices(self.n_qubits)
        pauli_set_n_qubits = nqubitspaulimatrices.get_pauli_matrices_of_n_qubits()
        
        nqubitsdepolarizingchannel = self.nqubitsdepolarizingchannel(self.n_qubits, pauli_set_n_qubits, p)
        kraus_matrices_of_a_depolarizing_channel = nqubitsdepolarizingchannel.get_kraus_matrices_of_a_depolarizing_channel()
        
        qml.QubitChannel(K_list=kraus_matrices_of_a_depolarizing_channel,
                         wires=[i for i in range(self.n_qubits)])
        
        qml.QubitChannel(kraus_matrices_of_a_pauli_channel,
                         wires=[i for i in range(self.n_qubits)])
        
        m = qml.PauliZ(0)
        for i in range(1, self.n_qubits):
            m = m @ qml.PauliZ(i)
        return qml.expval(m)
    
    
    def ufolding(self, noise_factor, operations, paras, p, kraus_matrices_of_a_pauli_channel):
    # This involves simulating the global unitary gate folding operation.
        nqubitspaulimatrices = self.nqubitspaulimatrices(self.n_qubits)
        pauli_set_n_qubits = nqubitspaulimatrices.get_pauli_matrices_of_n_qubits()
            
        nqubitsdepolarizingchannel = self.nqubitsdepolarizingchannel(self.n_qubits, pauli_set_n_qubits, p)
        kraus_matrices_of_a_depolarizing_channel = nqubitsdepolarizingchannel.get_kraus_matrices_of_a_depolarizing_channel()
        
        epoch = int((noise_factor - 1) / 2)
        
        for _ in range(epoch + 1):
            for i, operation in enumerate(operations):
                if operation == 'RX':
                    qml.RX(paras[i][1], wires=paras[i][0])
                elif operation == 'RY':
                    qml.RY(paras[i][1], wires=paras[i][0])
                elif operation == 'RZ':
                    qml.RZ(paras[i][1], wires=paras[i][0])
                elif operation == 'CNOT':
                    qml.CNOT(wires=[paras[i][0], paras[i][1]])
                    
            qml.QubitChannel(K_list=kraus_matrices_of_a_depolarizing_channel,
                             wires=[i for i in range(self.n_qubits)])
        for _ in range(epoch):
            for i, operation in enumerate(operations[::-1]):
                if operation == 'RX':
                    qml.RX(-paras[-i-1][1], wires=paras[-i-1][0])
                elif operation == 'RY':
                    qml.RY(-paras[-i-1][1], wires=paras[-i-1][0])
                elif operation == 'RZ':
                    qml.RZ(-paras[-i-1][1], wires=paras[-i-1][0])
                elif operation == 'CNOT':
                    qml.CNOT(wires=[paras[-i-1][0], paras[-i-1][1]])
            
            qml.QubitChannel(K_list=kraus_matrices_of_a_depolarizing_channel,
                             wires=[i for i in range(self.n_qubits)])   
            
        qml.QubitChannel(kraus_matrices_of_a_pauli_channel,
                         wires=[i for i in range(self.n_qubits)])
        
        m = qml.PauliZ(0)
        for i in range(1, self.n_qubits):
            m = m @ qml.PauliZ(i)
        return qml.expval(m)
    
    def calibration_cir1(self, kraus_matrices_of_a_pauli_channel):
        qml.QubitChannel(kraus_matrices_of_a_pauli_channel,
                         wires=[i for i in range(self.n_qubits)])
        m = qml.PauliZ(0)
        for i in range(1, self.n_qubits):
            m = m @ qml.PauliZ(i)
        return qml.expval(m)
    
    def calibration_cir2(self, kraus_matrices_of_a_pauli_channel):
        for i in range(self.n_qubits):
            qml.PauliX(wires=i)
        qml.QubitChannel(kraus_matrices_of_a_pauli_channel,
                         wires=[i for i in range(self.n_qubits)])
        m = qml.PauliZ(0)
        for i in range(1, self.n_qubits):
            m = m @ qml.PauliZ(i)
        return qml.expval(m)
    
    def qemzmsepc(self, noise_factor, operations, paras, p, kraus_matrices_of_a_pauli_channel, dev):
        z_unmitigated = qml.QNode(self.get_circuit, dev)(operations, paras, p, kraus_matrices_of_a_pauli_channel)
        # This is the output expectation value of the original noisy quantum circuit on the device dev.
        p_u = np.sqrt(qml.QNode(self.ufolding, dev)(noise_factor, operations, paras, p,
                                                    kraus_matrices_of_a_pauli_channel) / z_unmitigated)
        p_t = p_u * 0.5 * (np.abs(qml.QNode(self.calibration_cir1, dev)(kraus_matrices_of_a_pauli_channel)) +
                           np.abs(qml.QNode(self.calibration_cir2, dev)(kraus_matrices_of_a_pauli_channel)))
        z_mitigated = z_unmitigated / p_t
        return z_mitigated

In [7]:
# This represents the noise-free ideal quantum circuit.
@qml.qnode(dev)
def trotterstep1_ideal(rotation_angle_of_rx, rotation_angle_of_rz):
    qml.RX(rotation_angle_of_rx, wires=0)
    qml.RX(rotation_angle_of_rx, wires=1)
    qml.RX(rotation_angle_of_rx, wires=2)
    qml.RX(rotation_angle_of_rx, wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.RZ(rotation_angle_of_rz, wires=1)
    qml.RZ(rotation_angle_of_rz, wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[1, 2])
    qml.RZ(rotation_angle_of_rz, wires=2)
    qml.CNOT(wires=[1, 2])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1)@qml.PauliZ(2)@qml.PauliZ(3))

In [8]:
nqubitspaulimatrices = NqubitsPauliMatrices(n_qubits)
pauli_set_n_qubits = nqubitspaulimatrices.get_pauli_matrices_of_n_qubits()
# The following is a Pauli channel defined on N quantum bits that I randomly set up,
# representing measurement noise.
# Readers can adjust the parameters within this channel for additional simulation experiments.
kraus_matrices_of_a_pauli_channel = [np.sqrt(0.9)*pauli_set_n_qubits[0],
                      np.sqrt(0.02)*pauli_set_n_qubits[31],
                      np.sqrt(0.02)*pauli_set_n_qubits[150],
                      np.sqrt(0.02)*pauli_set_n_qubits[159],
                      np.sqrt(0.02)*pauli_set_n_qubits[160],
                      np.sqrt(0.02)*pauli_set_n_qubits[251]]

In [9]:
# In the simulation, we target different rotation angles of the RX gate.
# If the ZMSEPC theory is correct,
# the values after error mitigation through the QEM-ZMSEPC method should be the same as the ideal values.

rotation_angle = [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2.1, 2.3, 2.5, 2.7, 2.9, 3.1]
for rotation_angle_of_rx in rotation_angle:
    # The operations and paras required to simulate a Trotter step quantum circuit are as follows.
    operations = ['RX', 'RX', 'RX', 'RX', 'CNOT', 'CNOT', 'RZ', 'RZ', 'CNOT', 'CNOT', 'CNOT', 'RZ', 'CNOT']
    rotation_angle_of_rz = -0.2
    paras = [[0, rotation_angle_of_rx], [1, rotation_angle_of_rx], [2, rotation_angle_of_rx], [3, rotation_angle_of_rx], [0, 1], [2, 3], [1, rotation_angle_of_rz], [3, rotation_angle_of_rz], [0, 1], [2, 3], [1, 2], [2, rotation_angle_of_rz], [1, 2]]
    # apply QEM-ZMSEPC method
    qemzmsepc = QEMZMSEPC(n_qubits, NqubitsPauliMatrices, NqubitsDepolarizingChannel)
    noise_factor = 3
    z_mitigated = qemzmsepc.qemzmsepc(noise_factor, operations, paras, 0.93, kraus_matrices_of_a_pauli_channel, dev)
    print(trotterstep1_ideal(rotation_angle_of_rx, rotation_angle_of_rz) - z_mitigated)

7.105427357601002e-15
1.021405182655144e-14
-9.547918011776346e-15
4.440892098500626e-16
-5.995204332975845e-15
-2.060851489460447e-15
1.4051260155412137e-15
4.973777466277252e-16
2.409747748566282e-15
2.203098814490545e-15
-3.2890357104520263e-15
1.1102230246251565e-16
7.827072323607354e-15
8.992806499463768e-15
2.9976021664879227e-15
-2.3314683517128287e-15


In [10]:
# In all cases, it can be observed that the expectation values after error mitigation
# using the QEM-ZMSEPC method are identical to the ideal values, implying the validity of the ZMSEPC theory .